# Exploring Census ZCTA to County Relationship Data

This notebook explores the Census Bureau's ZCTA (Zip Code Tabulation Area) to County relationship file.

**Data Source:** [Census ZCTA County Relationship](https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt)

**Purpose:** Understanding how ZIP codes map to FIPS county codes


## 1. Import Libraries


In [1]:
import pandas as pd
import ssl
from urllib.request import urlopen
import io
import httpx


## 2. Configure SSL Context (for certificate issues)


In [2]:
# Disable SSL verification to avoid certificate errors
ssl._create_default_https_context = ssl._create_unverified_context


## 3. Load the Census Data


In [3]:
url = 'https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt'

print(f"Loading data from: {url}")
print("This may take a moment...")


Loading data from: https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt
This may take a moment...


### Method 1: Using httpx (recommended for SSL issues)


In [8]:
try:
    # Use httpx with SSL verification disabled
    response = httpx.get(url, timeout=60, verify=False)
    response.raise_for_status()
    
    # Read CSV from response text
    df = pd.read_csv(
        io.StringIO(response.text), 
        sep=',', 
        dtype={'ZCTA5': str, 'COUNTY': str, 'STATE': str}
    )
    
    print(f"✅ Successfully loaded {len(df):,} rows using httpx")
    
except Exception as e:
    print(f"❌ Error with httpx: {e}")
    print("Trying alternative method...")
    
    # Fallback: Use urlopen
    try:
        with urlopen(url) as response:
            df = pd.read_csv(
                response, 
                sep='|', 
                dtype={'ZCTA5': str, 'COUNTY': str, 'STATE': str}
            )
        print(f"✅ Successfully loaded {len(df):,} rows using urlopen")
    except Exception as e2:
        print(f"❌ Error with urlopen: {e2}")
        df = None


✅ Successfully loaded 44,410 rows using httpx


## 4. Explore the Data Structure


In [9]:
if df is not None:
    print("DataFrame Shape:", df.shape)
    print(f"\nColumns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print("\n" + "="*80)
    print("First few rows:")
    print("="*80)
    display(df.head(10))


DataFrame Shape: (44410, 24)

Columns (24):
   1. ZCTA5
   2. STATE
   3. COUNTY
   4. GEOID
   5. POPPT
   6. HUPT
   7. AREAPT
   8. AREALANDPT
   9. ZPOP
  10. ZHU
  11. ZAREA
  12. ZAREALAND
  13. COPOP
  14. COHU
  15. COAREA
  16. COAREALAND
  17. ZPOPPCT
  18. ZHUPCT
  19. ZAREAPCT
  20. ZAREALANDPCT
  21. COPOPPCT
  22. COHUPCT
  23. COAREAPCT
  24. COAREALANDPCT

First few rows:


,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,00601,72,001,72001,18465,7695,165132671,164333375,18570,7744,...,173777444,172725651,99.43,99.37,98.61,98.60,94.77,94.71,95.03,95.14
1,00601,72,141,72141,105,49,2326414,2326414,18570,7744,...,298027589,294039825,0.57,0.63,1.39,1.40,0.32,0.35,0.78,0.79
2,00602,72,003,72003,41520,18073,83734431,79288158,41520,18073,...,117948080,79904246,100.00,100.00,100.00,100.00,98.95,98.99,70.99,99.23
3,00603,72,005,72005,54689,25653,82063867,81880442,54689,25653,...,195741178,94608641,100.00,100.00,100.00,100.00,89.73,90.23,41.92,86.55
4,00606,72,093,72093,6276,2740,94864349,94851862,6615,2877,...,94864349,94851862,94.88,95.24,86.56,86.56,100.00,100.00,100.00,100.00
5,00606,72,121,72121,89,38,6679806,6679806,6615,2877,...,92831653,92806330,1.35,1.32,6.10,6.10,0.35,0.35,7.20,7.20
6,00606,72,153,72153,250,99,8048393,8048393,6615,2877,...,178241098,176615594,3.78,3.44,7.34,7.34,0.59,0.55,4.52,4.56
7,00610,72,003,72003,160,62,237185,237185,29016,12618,...,117948080,79904246,0.55,0.49,0.24,0.25,0.38,0.34,0.20,0.30
8,00610,72,011,72011,28856,12556,96956283,92784282,29016,12618,...,116355489,101747783,99.45,99.51,99.76,99.75,98.62,98.44,83.33,91.19
9,00612,72,013,72013,66938,30961,183876062,174066899,67010,30992,...,443385518,326200544,99.89,99.90,99.44,99.41,69.41,71.58,41.47,53.36


## 5. Understanding the Columns

**Key Columns for Zip-to-FIPS Mapping:**
- `ZCTA5`: 5-digit ZIP Code Tabulation Area (ZIP code)
- `STATE`: 2-digit state FIPS code
- `COUNTY`: 3-digit county FIPS code
- `GEOID`: Combined state+county FIPS code (5 digits)

**Population & Housing Data:**
- `POPPT`: Population in the intersection of ZCTA and County
- `HUPT`: Housing units in the intersection
- `ZPOP`: Total population in the ZCTA
- `COPOP`: Total population in the County

**Area Data:**
- `AREAPT`: Area of intersection
- `ZAREA`: Total area of ZCTA
- `COAREA`: Total area of County

**Percentage Columns:**
- `ZPOPPCT`: % of ZCTA population in this county
- `COPOPPCT`: % of County population in this ZCTA


## 6. Create Zip-to-FIPS Mapping


In [6]:
if df is not None:
    # Create FIPS code from STATE + COUNTY
    df['FIPS'] = (df['STATE'].astype(str).str.zfill(2) + df['COUNTY'].astype(str).str.zfill(3))
    
    # Ensure ZIP codes are 5 digits
    df['ZIP'] = df['ZCTA5'].astype(str).str.zfill(5)
    
    # Create simplified mapping table
    zip_to_fips = df[['ZIP', 'FIPS', 'STATE', 'COUNTY', 'ZPOPPCT', 'COPOPPCT']].copy()
    
    print(f"Total ZIP-to-FIPS relationships: {len(zip_to_fips):,}")
    print(f"Unique ZIP codes: {zip_to_fips['ZIP'].nunique():,}")
    print(f"Unique FIPS codes: {zip_to_fips['FIPS'].nunique():,}")
    
    print("\n" + "="*80)
    print("Sample Zip-to-FIPS Mapping:")
    print("="*80)
    display(zip_to_fips.head(20))


KeyError: 'STATE'

## 7. Understanding Multiple Counties per ZIP Code

Some ZIP codes span multiple counties. This is important to understand when aggregating data!


In [ ]:
if df is not None:
    # Find ZIP codes that span multiple counties
    zip_counts = zip_to_fips.groupby('ZIP')['FIPS'].count().reset_index()
    zip_counts.columns = ['ZIP', 'county_count']
    
    multi_county_zips = zip_counts[zip_counts['county_count'] > 1]
    
    print(f"ZIP codes in single county: {len(zip_counts[zip_counts['county_count'] == 1]):,}")
    print(f"ZIP codes spanning multiple counties: {len(multi_county_zips):,}")
    print(f"\nMax counties per ZIP: {zip_counts['county_count'].max()}")
    
    print("\n" + "="*80)
    print("Examples of ZIP codes spanning multiple counties:")
    print("="*80)
    
    # Show examples
    example_zips = multi_county_zips.nlargest(5, 'county_count')['ZIP'].tolist()
    for zip_code in example_zips[:3]:  # Show first 3 examples
        zip_data = zip_to_fips[zip_to_fips['ZIP'] == zip_code][['ZIP', 'FIPS', 'ZPOPPCT', 'COPOPPCT']]
        print(f"\nZIP {zip_code} spans {len(zip_data)} counties:")
        display(zip_data)


## 8. Create Final Mapping Table (for use in applications)


In [ ]:
if df is not None:
    # Option 1: Keep all relationships (ZIP can map to multiple FIPS)
    mapping_all = zip_to_fips[['ZIP', 'FIPS']].drop_duplicates()
    
    # Option 2: Keep only primary county (where ZIP has highest population %)
    mapping_primary = zip_to_fips.sort_values('ZPOPPCT', ascending=False).groupby('ZIP').first()
    mapping_primary = mapping_primary[['FIPS']].reset_index()
    
    print("Option 1: All ZIP-to-FIPS relationships")
    print(f"  Rows: {len(mapping_all):,}")
    print(f"  Unique ZIPs: {mapping_all['ZIP'].nunique():,}")
    
    print("\nOption 2: Primary county only (highest population %)")
    print(f"  Rows: {len(mapping_primary):,}")
    print(f"  Unique ZIPs: {mapping_primary['ZIP'].nunique():,}")
    
    print("\n" + "="*80)
    print("Sample of Primary Mapping:")
    print("="*80)
    display(mapping_primary.head(20))


## 9. Example: Find ZIP codes for a specific FIPS code


In [ ]:
if df is not None:
    # Example: Find all ZIP codes in Miami-Dade County, Florida
    # Miami-Dade FIPS: 12025 (State 12, County 025)
    example_fips = '12025'
    
    zip_codes_in_county = zip_to_fips[zip_to_fips['FIPS'] == example_fips]['ZIP'].unique()
    
    print(f"FIPS Code: {example_fips}")
    print(f"Number of ZIP codes in this county: {len(zip_codes_in_county)}")
    print(f"\nFirst 20 ZIP codes: {', '.join(sorted(zip_codes_in_county)[:20])}...")
    
    # Show detailed data
    print("\n" + "="*80)
    print(f"Detailed data for FIPS {example_fips}:")
    print("="*80)
    display(zip_to_fips[zip_to_fips['FIPS'] == example_fips].head(10))


## 10. Example: Find FIPS codes for a specific ZIP code


In [ ]:
if df is not None:
    # Example: Find counties for ZIP code 10001 (Manhattan)
    example_zip = '10001'
    
    fips_for_zip = zip_to_fips[zip_to_fips['ZIP'] == example_zip]
    
    print(f"ZIP Code: {example_zip}")
    print(f"Number of counties this ZIP spans: {len(fips_for_zip)}")
    
    print("\n" + "="*80)
    print(f"Counties for ZIP {example_zip}:")
    print("="*80)
    display(fips_for_zip[['ZIP', 'FIPS', 'STATE', 'COUNTY', 'ZPOPPCT', 'COPOPPCT']])


## 11. Summary Statistics


In [ ]:
if df is not None:
    print("="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    
    print(f"\nTotal records: {len(df):,}")
    print(f"Unique ZIP codes (ZCTA5): {df['ZCTA5'].nunique():,}")
    print(f"Unique FIPS codes: {zip_to_fips['FIPS'].nunique():,}")
    print(f"Unique States: {df['STATE'].nunique()}")
    
    zip_counts = zip_to_fips.groupby('ZIP')['FIPS'].count().reset_index()
    zip_counts.columns = ['ZIP', 'county_count']
    
    print(f"\nZIP codes in single county: {len(zip_counts[zip_counts['county_count'] == 1]):,}")
    print(f"ZIP codes spanning 2 counties: {len(zip_counts[zip_counts['county_count'] == 2]):,}")
    print(f"ZIP codes spanning 3+ counties: {len(zip_counts[zip_counts['county_count'] >= 3]):,}")
    
    print(f"\nAverage counties per ZIP: {zip_counts['county_count'].mean():.2f}")
    print(f"Median counties per ZIP: {zip_counts['county_count'].median():.0f}")
    print(f"Max counties per ZIP: {zip_counts['county_count'].max()}")
